<a href="https://colab.research.google.com/github/VardhanYadav/Movie-Recommendation-System-Using-Information-Retrieval/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install Kaggle
from google.colab import drive
drive.mount('/content/drive')
!kaggle datasets download -d tmdb/tmdb-movie-metadata
from zipfile import ZipFile
import os

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
 79% 7.00M/8.89M [00:00<00:00, 69.3MB/s]
100% 8.89M/8.89M [00:00<00:00, 81.6MB/s]


In [ ]:
# Path to the downloaded dataset zip file
dataset = '/content/tmdb-movie-metadata.zip'

In [ ]:
# Check if the zip file exists before attempting to extract
if os.path.exists(dataset):
    with ZipFile(dataset, 'r') as zip:
        zip.extractall('tmdb-movie-metadata')  # Specify a directory for extraction
        print('The dataset is extracted to "tmdb-movie-metadata" directory')
else:
    print(f'The dataset file "{dataset}" does not exist.')

The dataset is extracted to "tmdb-movie-metadata" directory


In [ ]:
import pandas as pd
import os


In [ ]:
# Define the path to the extracted dataset directory
data_dir = 'tmdb-movie-metadata'  # This should match the extraction directory name

# Construct the full paths to the CSV files
movies_path = os.path.join(data_dir, 'tmdb_5000_movies.csv')
credits_path = os.path.join(data_dir, 'tmdb_5000_credits.csv')


In [ ]:
# Load datasets into DataFrames
if os.path.exists(movies_path) and os.path.exists(credits_path):
    movies = pd.read_csv(movies_path)
    credits = pd.read_csv(credits_path)

    # Display the first few rows of each DataFrame to confirm successful loading
    print("Movies DataFrame:")
    print(movies.head())

    print("\nCredits DataFrame:")
    print(credits.head())
else:
    print("One or both of the CSV files do not exist in the specified directory.")
movies.head(2)
movies.shape
credits.head()
movies = movies.merge(credits,on='title')
movies.head()
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head()
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
movies.head()
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L
movies['cast'] = movies['cast'].apply(convert)
movies.head()
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
movies['crew'] = movies['crew'].apply(fetch_director)
movies.sample(5)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
movies.head()
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

vector = cv.fit_transform(new['tags']).toarray()

vector.shape
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity
new[new['title'] == 'The Lego Movie'].index[0]

def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)


recommend('Gandhi')

Movies DataFrame:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2 

In [ ]:
import pickle

pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import pickle
import pandas as pd
from pathlib import Path

def load_model_files():
    """
    Safely load model files with error handling
    """
    try:
        movies = pickle.load(open('movie_list.pkl', 'rb'))
        similarity = pickle.load(open('similarity.pkl', 'rb'))
        return movies, similarity
    except FileNotFoundError:
        raise Exception("Required model files not found. Please ensure 'movie_list.pkl' and 'similarity.pkl' are in the same directory.")
    except Exception as e:
        raise Exception(f"Error loading model files: {str(e)}")

# Load the models
try:
    movies, similarity = load_model_files()
except Exception as e:
    print(f"Error: {str(e)}")
    exit(1)

def get_movie_suggestions(query):
    """
    Get movie suggestions as user types
    """
    if not query:
        return []
    suggestions = movies[movies['title'].str.contains(query, case=False, na=False)]['title'].tolist()
    return suggestions[:5]

def fetch_recommendations(movie_name):
    """
    Fetch movie recommendations based on the input movie name with enhanced error handling and formatting
    """
    try:
        # Check if movie name is empty
        if not movie_name:
            return "Please enter a movie name."

        # Find the movie in the database
        movie_matches = movies[movies['title'].str.lower() == movie_name.lower()]

        if movie_matches.empty:
            close_matches = movies[movies['title'].str.contains(movie_name, case=False, na=False)]['title'].tolist()[:5]
            if close_matches:
                suggestions = "\n• ".join(close_matches)
                return f"Movie not found. Did you mean one of these?\n• {suggestions}"
            return "Movie not found. Please check the spelling or try another movie."

        # Get the index and find similar movies
        index = movie_matches.index[0]
        distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])

        # Format recommendations with similarity scores
        recommendations = []
        for i in distances[1:6]:
            movie = movies.iloc[i[0]]
            similarity_score = i[1] * 100  # Convert to percentage
            recommendations.append(f"• {movie.title} (Similarity: {similarity_score:.1f}%)")

        return "Recommended movies:\n" + "\n".join(recommendations)

    except Exception as e:
        return f"An error occurred: {str(e)}"

def create_interface():
    """
    Create and configure the Gradio interface with improved styling
    """
    with gr.Blocks(theme=gr.themes.Soft()) as iface:
        gr.Markdown(
            """
            # 🎬 Movie Recommendation System

            Enter the name of a movie you like, and get 5 similar movies recommended to you!
            The recommendations are based on movie features like genre, cast, crew, and plot keywords.
            """
        )

        with gr.Row():
            with gr.Column(scale=2):
                movie_input = gr.Textbox(
                    label="Enter Movie Name",
                    placeholder="Start typing a movie name...",
                    max_lines=1
                )

        with gr.Row():
            with gr.Column():
                output = gr.TextArea(
                    label="Recommendations",
                    interactive=False,
                    lines=8
                )

        # Add examples
        gr.Examples(
            examples=[
                ["The Dark Knight"],
                ["Avatar"],
                ["Inception"],
                ["The Godfather"],
                ["Interstellar"]
            ],
            inputs=movie_input,
            outputs=output,
            fn=fetch_recommendations,
            cache_examples=True
        )

        # Set up event handlers
        movie_input.submit(fn=fetch_recommendations, inputs=movie_input, outputs=output)
        movie_input.change(fn=get_movie_suggestions, inputs=movie_input, outputs=movie_input)

    return iface

# Launch the interface
if __name__ == "__main__":  # Fixed the main check syntax
    iface = create_interface()
    iface.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        debug=True
    )

Caching examples at: '/content/.gradio/cached_examples/72'
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://af95d5c8ca92f77c21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2044, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1591, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://af95d5c8ca92f77c21.gradio.live
